## Welcome to your first casestudy
- In this case study you have to scrape weather data from the website  **"http://www.estesparkweather.net/archive_reports.php?date=200901"**
- Scrape all the available attributes of weather data for each day from **2009-01-01 to 2018-10-28**
- Ignore records for missing days
- Represent the scraped data as **pandas dataframe** object.

### Dataframe specific deatails
- Expected column names (order dose not matter):   
       ['Average temperature (°F)', 'Average humidity (%)',
       'Average dewpoint (°F)', 'Average barometer (in)',
       'Average windspeed (mph)', 'Average gustspeed (mph)',
       'Average direction (°deg)', 'Rainfall for month (in)',
       'Rainfall for year (in)', 'Maximum rain per minute',
       'Maximum temperature (°F)', 'Minimum temperature (°F)',
       'Maximum humidity (%)', 'Minimum humidity (%)', 'Maximum pressure',
       'Minimum pressure', 'Maximum windspeed (mph)',
       'Maximum gust speed (mph)', 'Maximum heat index (°F)']
- Each record in the dataframe corresponds to weather deatils of a given day
- Make sure the index column is **date-time format (yyyy-mm-dd)**
- Perform necessary data cleaning and type cast each attributes to relevent data type

### Saving the dataframe
- Once you are done with you scrapping save your dataframe as pickle file by name 'dataframe.pk'

#### Sample code to save pickle file
```python
import pickle
with open("dataframe.pk", "wb") as file:
    pickle.dump(<your_dataframe>, file)
```
 
 

### Run the below cell to import necessary packages
- These packages should be sufficient to perform you task
- In case if you are looking are any other packages run **!pip3 install <package_name> --user with in a cell**

In [1]:
import bs4
from bs4 import BeautifulSoup
import csv
import requests
import time
import pandas as pd
import urllib
import re
import pickle

In [2]:
import calendar

## Scraping data from website

In [3]:
#### Start you code here, you are free to add any number of cells

start_month = 1
start_year  = 2009
end_month   = 10
end_year    = 2018
    
fetch = False
data = []

for i in range(int(start_year),int(end_year+1)):
    year = i
    
    if i == end_year:
        endmon = end_month+1
    else:
        endmon = 12+1
        
    for mon in range(1,endmon):
        if mon == start_month:
            fetch = True
        if fetch:
            mon_num = '{:02d}'.format(mon)
            mon_abb = calendar.month_abbr[mon]
            mon_day = calendar.monthrange(year,mon)[1]

            url = 'http://www.estesparkweather.net/archive_reports.php?date='+str(year)+mon_num
            res = requests.get(url).text
            soup = bs4.BeautifulSoup(res,'lxml')

# Extracting Table data from a web page

            table = soup.find_all('table')

            rows = []
            for day in table:
                row = day.find_all('tr')
                rows.append(row)
            #rows = rows[:mon_day]  

            cols = []
            for row in rows:
                for x in row:
                    col = x.find_all('td')
                    cols.append(col)

            temp={}
            ind=0
            for y in cols:
                if ind == 0:
                    try:
                        date = int(y[0].get_text()[4:6].strip())
                        date = str(year) + '-' + mon_num + '-' +'{:02d}'.format(date)
                        ind+=1
                    except ValueError:
                        break
                else:
                    var=y[0].get_text()
                    val=y[1].get_text()
                    temp[var]=val
                    ind+=1

                if ind==20:
                    ind=0
                    data.append([date,temp]) 
                    temp={}

## Converting scraped data into DataFrame object & aligning name of columns as per requirement

In [4]:
# As the data is required till 2018-10-28 and October has 31 days and hence removing last 3 records as data[:-3]

index  = [pd.to_datetime(i[0]) for i in data[:-3]]
values = [i[1] for i in data[:-3]]

In [5]:
cols =['Average temperature', 'Average humidity',
       'Average dewpoint', 'Average barometer',
       'Average windspeed', 'Average gustspeed',
       'Average direction', 'Rainfall for month',
       'Rainfall for year', 'Maximum rain per minute',
       'Maximum temperature', 'Minimum temperature',
       'Maximum humidity', 'Minimum humidity', 'Maximum pressure',
       'Minimum pressure', 'Maximum windspeed',
       'Maximum gust speed', 'Maximum heat index']

In [6]:
weather_df = pd.DataFrame(values,index=index)
weather_df = weather_df[cols]
weather_df.tail()

,Average temperature,Average humidity,Average dewpoint,Average barometer,Average windspeed,Average gustspeed,Average direction,Rainfall for month,Rainfall for year,Maximum rain per minute,Maximum temperature,Minimum temperature,Maximum humidity,Minimum humidity,Maximum pressure,Minimum pressure,Maximum windspeed,Maximum gust speed,Maximum heat index
2018-10-24,45.9°F,53%,27.5°F,29.8 in.,2.7 mph,4.6 mph,282° (WNW),0.85 in.,6.86 in.,0.00 in. on day 24 at time 23:59,57.3°F on day 24 at time 13:48,35.9°F on day 24 at time 23:50,81% on day 24 at time 04:44,23% on day 24 at time 13:48,29.947 in. on day 24 at time 08:15,29.710 in. on day 24 at time 15:05,12.7 mph on day 24 at time 17:14,19.6 mph from 283 °(WNW) on day 24 at time 1...,57.2°F on day 24 at time 13:48
2018-10-25,46.7°F,40%,22.7°F,29.8 in.,4.5 mph,7.2 mph,283° (WNW),0.85 in.,6.86 in.,0.00 in. on day 25 at time 23:59,61.8°F on day 25 at time 14:58,35.0°F on day 25 at time 01:02,60% on day 25 at time 21:16,19% on day 25 at time 14:59,29.933 in. on day 25 at time 08:13,29.682 in. on day 25 at time 15:00,13.8 mph on day 25 at time 21:55,19.6 mph from 283 °(WNW) on day 25 at time 2...,61.8°F on day 25 at time 14:58
2018-10-26,48.4°F,35%,21.6°F,29.8 in.,9.7 mph,14.4 mph,283° (WNW),0.85 in.,6.86 in.,0.00 in. on day 26 at time 23:59,58.7°F on day 26 at time 13:42,42.6°F on day 26 at time 04:33,48% on day 26 at time 04:04,19% on day 26 at time 13:42,29.979 in. on day 26 at time 09:05,29.759 in. on day 26 at time 00:26,20.7 mph on day 26 at time 16:35,31.1 mph from 283 °(WNW) on day 26 at time 0...,58.7°F on day 26 at time 13:42
2018-10-27,53.3°F,37%,26.6°F,29.8 in.,10.3 mph,14.8 mph,282° (WNW),0.85 in.,6.86 in.,0.00 in. on day 27 at time 23:59,64.5°F on day 27 at time 15:13,38.3°F on day 27 at time 23:59,78% on day 27 at time 23:59,24% on day 27 at time 16:39,29.935 in. on day 27 at time 23:59,29.650 in. on day 27 at time 15:11,28.8 mph on day 27 at time 06:40,38.0 mph from 283 °(WNW) on day 27 at time 0...,64.5°F on day 27 at time 15:13
2018-10-28,49.4°F,48%,25.2°F,29.9 in.,5.3 mph,7.6 mph,282° (WNW),0.85 in.,6.86 in.,0.00 in. on day 28 at time 23:59,67.3°F on day 28 at time 13:57,30.9°F on day 28 at time 05:41,90% on day 28 at time 06:15,19% on day 28 at time 13:57,30.040 in. on day 28 at time 07:46,29.663 in. on day 28 at time 23:59,18.4 mph on day 28 at time 13:13,25.3 mph from 260 °( W ) on day 28 at time 1...,67.3°F on day 28 at time 13:57


In [7]:
new_cols = ['Average temperature (°F)', 'Average humidity (%)',
           'Average dewpoint (°F)', 'Average barometer (in)',
           'Average windspeed (mph)', 'Average gustspeed (mph)',
           'Average direction (°deg)', 'Rainfall for month (in)',
           'Rainfall for year (in)', 'Maximum rain per minute',
           'Maximum temperature (°F)', 'Minimum temperature (°F)',
           'Maximum humidity (%)', 'Minimum humidity (%)', 'Maximum pressure',
           'Minimum pressure', 'Maximum windspeed (mph)',
           'Maximum gust speed (mph)', 'Maximum heat index (°F)']

In [8]:
weather_df.columns = new_cols
weather_df.head()

,Average temperature (°F),Average humidity (%),Average dewpoint (°F),Average barometer (in),Average windspeed (mph),Average gustspeed (mph),Average direction (°deg),Rainfall for month (in),Rainfall for year (in),Maximum rain per minute,Maximum temperature (°F),Minimum temperature (°F),Maximum humidity (%),Minimum humidity (%),Maximum pressure,Minimum pressure,Maximum windspeed (mph),Maximum gust speed (mph),Maximum heat index (°F)
2009-01-01,37.8 °F,35 %,12.7 °F,29.7 in.,26.4 mph,36.8 mph,274 ° ( W ),0.00 in.,0.00 in.,0.00 in. on day 02 at time 00:00,40.1 °F on day 01 at time 13:48,34.5 °F on day 01 at time 03:45,44 % on day 01 at time 22:37,27 % on day 00 at time 00:28,29.762 in. on day 01 at time 09:14,29.596 in. on day 01 at time 01:29,41.4 mph on day 01 at time 11:27,59 mph from 270 °( W ) on day 01 at time 21:15,40.1 °F on day 01 at time 13:48
2009-01-02,43.2 °F,32 %,14.7 °F,29.5 in.,12.8 mph,18.0 mph,240 ° (WSW),0.00 in.,0.00 in.,0.00 in. on day 02 at time 23:57,52.8 °F on day 02 at time 12:37,37.5 °F on day 01 at time 00:50,43 % on day 01 at time 00:15,16 % on day 02 at time 12:13,29.669 in. on day 01 at time 00:44,29.268 in. on day 02 at time 23:44,35.7 mph on day 02 at time 01:55,51 mph from 270 °( W ) on day 02 at time 06:34,52.8 °F on day 02 at time 12:37
2009-01-03,25.7 °F,60 %,12.7 °F,29.7 in.,8.3 mph,12.2 mph,290 ° (WNW),0.00 in.,0.00 in.,0.00 in. on day 03 at time 23:57,41.2 °F on day 02 at time 00:11,6.7 °F on day 03 at time 23:57,89 % on day 03 at time 06:54,35 % on day 03 at time 02:56,30.232 in. on day 03 at time 23:29,29.260 in. on day 03 at time 01:29,25.3 mph on day 03 at time 02:44,38 mph from 248 °(WSW) on day 02 at time 00:05,41.2 °F on day 02 at time 00:11
2009-01-04,9.3 °F,67 %,0.1 °F,30.4 in.,2.9 mph,4.5 mph,47 ° ( NE),0.00 in.,0.00 in.,0.00 in. on day 04 at time 23:57,19.4 °F on day 04 at time 23:03,-0.0 °F on day 04 at time 21:00,79 % on day 04 at time 22:12,35 % on day 04 at time 23:19,30.566 in. on day 04 at time 09:59,30.227 in. on day 03 at time 00:00,12.7 mph on day 04 at time 23:55,20 mph from 248 °(WSW) on day 04 at time 23:33,32.0 °F on day 00 at time 00:00
2009-01-05,23.5 °F,30 %,-5.3 °F,29.9 in.,16.7 mph,23.1 mph,265 ° ( W ),0.00 in.,0.00 in.,0.00 in. on day 05 at time 23:57,30.3 °F on day 05 at time 13:27,15.1 °F on day 04 at time 02:50,56 % on day 05 at time 20:38,13 % on day 05 at time 04:16,30.233 in. on day 04 at time 00:00,29.568 in. on day 05 at time 16:14,38.0 mph on day 05 at time 13:45,53 mph from 248 °(WSW) on day 05 at time 14:33,32.0 °F on day 00 at time 00:00


## Cleaning Data

In [9]:
weather_df['Average temperature (°F)']= weather_df['Average temperature (°F)'].apply(lambda x:float(x.split('°F')[0].strip()))    
weather_df['Average humidity (%)']    = weather_df['Average humidity (%)'].apply(lambda x:float(x.split('%')[0].strip()))
weather_df['Average dewpoint (°F)']   = weather_df['Average dewpoint (°F)'].apply(lambda x:float(x.split('°F')[0].strip()))
weather_df['Average barometer (in)']  = weather_df['Average barometer (in)'].apply(lambda x:float(x.split('in.')[0].strip()))

weather_df['Average windspeed (mph)'] = weather_df['Average windspeed (mph)'].apply(lambda x:float(x.split('mph')[0].strip()))
weather_df['Average gustspeed (mph)'] = weather_df['Average gustspeed (mph)'].apply(lambda x:float(x.split('mph')[0].strip()))

weather_df['Average direction (°deg)']= weather_df['Average direction (°deg)'].apply(lambda x:int(x.split('°')[0].strip()))
weather_df['Rainfall for month (in)'] = weather_df['Rainfall for month (in)'].apply(lambda x:float(x.split('in.')[0].strip()))
weather_df['Rainfall for year (in)']  = weather_df['Rainfall for year (in)'].apply(lambda x:float(x.split('in.')[0].strip()))
weather_df['Maximum rain per minute'] = weather_df['Maximum rain per minute'].apply(lambda x:float(x.split('in.')[0].strip()))

weather_df['Maximum temperature (°F)'] = weather_df['Maximum temperature (°F)'].apply(lambda x:float(x.split('°F')[0].strip()))
weather_df['Minimum temperature (°F)'] = weather_df['Minimum temperature (°F)'].apply(lambda x:float(x.split('°F')[0].strip()))

weather_df['Maximum humidity (%)'] = weather_df['Maximum humidity (%)'].apply(lambda x:float(x.split('%')[0].strip()))
weather_df['Minimum humidity (%)'] = weather_df['Minimum humidity (%)'].apply(lambda x:float(x.split('%')[0].strip()))

weather_df['Maximum pressure']     = weather_df['Maximum pressure'].apply(lambda x:float(x.split('in.')[0].strip()))
weather_df['Minimum pressure']     = weather_df['Minimum pressure'].apply(lambda x:float(x.split('in.')[0].strip()))

weather_df['Maximum windspeed (mph)'] = weather_df['Maximum windspeed (mph)'].apply(lambda x:float(x.split('mph')[0].strip()))
weather_df['Maximum gust speed (mph)'] = weather_df['Maximum gust speed (mph)'].apply(lambda x:float(x.split('mph')[0].strip()))

weather_df['Maximum heat index (°F)'] = weather_df['Maximum heat index (°F)'].apply(lambda x:float(x.split('°F')[0].strip()))

In [10]:
with open("dataframe.pk", "wb") as file:
    pickle.dump(weather_df, file)